In [ ]:
import os
import shutil
from creds import OPENAI_KEY
from application_generator import JobApplicationBuild, JobApplicationBuilder
from utils import LatexToolBox, text_to_pdf
from models import ChatGPT
from prompts.resume_section_prompts import RESUME_WRITER_PERSONA
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

llm = ChatGPT(
    api_key=OPENAI_KEY,
    model="gpt-4o",
    system_prompt=RESUME_WRITER_PERSONA,
    max_output_tokens=None,
    temperature=0.7
)

build = JobApplicationBuild(
    resume_tex_template_path="slightly_less_basic_template.tex",
    job_content_path=r"input_data/Intuitive Surgical Data Engineer Intern.pdf",
    user_details_content_path=r"input_data/full_resume_contents_20250122.pdf"
)

generator = JobApplicationBuilder(
    llm=llm
)

build = generator.extract_job_content(build)
build = generator.user_data_extraction(build)
build = generator.generate_resume_json(build)
build = generator.resume_json_to_resume_tex(build)
print("Done generating resume tex file")

custom_letter_instructions = input("Enter any custom application instructions for the cover letter: ")
build = generator.generate_cover_letter(build, custom_letter_instructions, need_pdf=False)

# check that the fonts in the resume tex are available and flag the ones not installed
resume_tex_fonts, _ = LatexToolBox.extract_tex_font_dependencies(build.resume_tex_path)
font_statuses = LatexToolBox.check_fonts_installed(resume_tex_fonts)
if not all([v for k, v in font_statuses.items()]):
    [print(f"{k} not installed") for k, v in font_statuses.items() if not v]

generator.cleanup_files(build)


Edit the resume latex and cover letter text before running the next cell.

In [ ]:
LatexToolBox.compile_latex_to_pdf(tex_file_path=build.resume_tex_path)
print(f"Resume PDF is saved at {build.resume_tex_path.replace('.tex', '.pdf')}")

cover_letter_pdf_path = build.cover_letter_path.replace('.txt', '.pdf')
cover_letter_pdf_path = text_to_pdf(build.cover_letter_path, cover_letter_pdf_path)
print(f"Cover Letter PDF is saved at {cover_letter_pdf_path}")

In [ ]:
# move the job content file content to the output folder
#shutil.move(build.job_content_path, os.path.join("output", os.path.basename(build.job_content_path)))

Edit the resume latex and cover letter text before running the next cell.

In [ ]:
LatexToolBox.compile_latex_to_pdf(tex_file_path=resume_tex_path)
print(f"Resume PDF is saved at {resume_tex_path.replace('.tex', '.pdf')}")

cover_letter_pdf_path = cover_letter_txt_path.replace('.txt', '.pdf')
cover_letter_pdf_path = text_to_pdf(cover_letter, cover_letter_pdf_path)
print(f"Cover Letter PDF is saved at {cover_letter_pdf_path}")

In [ ]:
# move the job content file to the output folder
shutil.move(job_content_path, generator.get_job_doc_path())

In [ ]:
import os
import shutil
from creds import OPENAI_KEY
from application_generator import JobApplicationBuilder
from application_generator import JobApplicationBuilder
from utils import LatexToolBox, text_to_pdf
from models import ChatGPT
from prompts.resume_section_prompts import RESUME_WRITER_PERSONA
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

LatexToolBox.compile_latex_to_pdf(tex_file_path='output_files/Paypal/DataScientistIn_20250122195204/resume.tex')